### Evita rindani sembring
### 12S15066

In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math
import string
import re

### Preprocessing Document

In [2]:
file = open('collections/trec.sample.xml', 'r', encoding = 'utf-8')
data = []

In [3]:
for line in file.readlines():
    data.append(line)

from xml.etree.ElementTree import ElementTree
tree = ElementTree()
tree.parse("collections/trec.sample.xml")

<Element 'HEAD' at 0x000001789A444D18>

In [4]:
docnum = []
headline = []
text = []

for node in tree.iter("DOCNO"):
    docnum.append(node.text)
    
for node in tree.iter("HEADLINE"):
    headline.append(node.text)

for node in tree.iter("TEXT"):
    text.append(node.text)
    
sums = []
for i in range(len(text)):
    sums.append(headline[i] + text[i])
    
sums[0]
print(docnum)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '

In [5]:
tokenize = []
translator = str.maketrans('','',string.punctuation)

In [6]:
for i in range(len(sums)):
    sums[i] = sums[i].translate(translator)
    sums[i] = ''.join([x for x in sums[i] if not x.isdigit()])
    sums[i] = re.sub('r ^ https ?:\/\/.*[\r\n]*','',sums[i],flags = re.MULTILINE)
    tokenize.append(word_tokenize(sums[i]))

In [7]:
for i in range(len(tokenize)):
    for word in tokenize[i]:
        tokenize[i] = [word.lower() for word in tokenize[i]]

In [8]:
datalist = []
for i in range(len(tokenize)):
    newList = [w for w in tokenize[i] if not w in stopwords.words('english')]
    datalist.append(newList)

In [9]:
stemmer = PorterStemmer()
dataStemList = []
unique = []

In [10]:
for i in range(len(datalist)):
    news = []
    for word in datalist[i]:
        if(word != stemmer.stem(word)):
            word = stemmer.stem(word)
            news.append(word)
        else:
            news.append(word)
        if not word in unique:
            unique.append(word)
            
    dataStemList.append(news)
    del news

In [11]:
files = open('collections/queries.lab3.txt', 'r', encoding = 'utf-8')

In [12]:
datas = []
for i in files.readlines():
    datas.append(i)
    
query = []
for x in datas:
    w = x.split()
    l = []
    for word in w:
        if not word.isdigit():
            l.append(word)
    query.append(l)

### Preprocessing Query

In [13]:
#Casefolding
for i in range(len(query)):
    query[i] = [word.lower() for word in query[i]]
     
#Stopping
for i in range(len(query)):
    query[i] = [w for w in query[i] if not w in stopwords.words('english')]
    
    
#stemming
for i in range(len(query)):
    query[i] = [stemmer.stem(word) if word != stemmer.stem(word) else word for word in query[i]]
    
    
uniq = []
for i in range(len(query)):
    for word in query[i]:
        if not word in uniq:
            uniq.append(word)

### TFIDF Implementation using <i>Term Weighting</i>

In [14]:
N = len(dataStemList)
df = []
res = []
wtd = []
hasil = []

In [15]:
for i in range(len(uniq)):
    tot = 0
    for j in range(len(dataStemList)):
        if uniq[i] in dataStemList[j]:
            tot += 1
    df.append(tot)
    
for i in range(len(df)):
    res.append(math.log10(N/df[i]))

In [16]:
for i in range(len(uniq)):
    lists = []
    for j in range(len(dataStemList)):
        dicts = {}
        x = dataStemList[j].count(uniq[i])
        if x == 0:
            dicts[j+1] = 0
            lists.append(dicts)
        else:
            score = math.log10(x)
            score += 1
            score *= res[i]
            dicts[j+1] = score
            lists.append(dicts)
    wtd.append(lists)

In [17]:
for i in range(len(query)):
    l = []
    for j in range(len(dataStemList)):
        dic = {}
        for word in query[i]:
            sums = 0
            ind = uniq.index(word)
            #print(ind)
            for val in wtd[ind][j].values():
                sums += val
        if(sums!= 0):
            dic['docnum'] = j+1
            dic['score'] = sums
        if(len(dic) != 0): l.append(dic)
    hasil.append(l)

In [18]:
for i in range(len(query)):
    hasil[i] = sorted(hasil[i], key = lambda x : x['score'], reverse = True)

In [19]:
for i in range(len(query)):
    with open('Tugas1A-{counter}.txt'.format(counter = i+1), 'w') as f:
        f.write('First 10 Documents:\n')
        f.write('Q_ID | DOCNO | SCORE\n')
        if len(hasil[i]) > 10:
            for x in range(10):
                c = i + 1
                f.write('%s   0   %s   0   %s\n' %(c,docnum[hasil[i][x]['docnum']-1],hasil[i][x]['score']))
        else:
            for x in hasil[i]:
                c  = i + 1
                f.write('%s   0   %s   0   %s\n' %(c,docnum[x['docnum']-1],x['score']))

### TFIDF Implementation using <i>Smart Notation</i>

In [20]:
dfq = []
resq = []
wtq = []
new_wtd = []
norm = []
norms = []   
rescosine = []

In [21]:
for i in range(len(uniq)):
    s = 0
    for x in range(len(query)):
        if uniq[i] in query[x]:
            s += 1
    dfq.append(s)

In [22]:
for i in range(len(dfq)):
    resq.append(math.log10(N / dfq[i]))

In [23]:
for i in range(len(uniq)):
    lists = []
    for j in range(len(query)):
        dicts = {}
        x = query[j].count(uniq[i])
        if x == 0:
            dicts[j+1] = 0
            lists.append(dicts)
        else:
            score = math.log10(x)
            score += 1
            score *= resq[i]
            dicts[j+1] = score
            lists.append(dicts)
    wtq.append(lists)    

In [24]:
for i in range(len(uniq)):
    lists = []
    for j in range(len(dataStemList)):
        dicts = {}
        x = dataStemList[j].count(uniq[i])
        if x == 0:
            dicts[j+1] = 0
            lists.append(dicts)
        else:
            score = math.log10(x)
            score += 1
            dicts[j+1] = score  
            lists.append(dicts)
    new_wtd.append(lists)

In [25]:
for i in range(len(uniq)):
    ss = []
    g = 0
    for x in wtq[i]:
        for val in x.values():
            if val!=0: ss.append(val)
    #print(ss)
    for c in ss:
        g = g + math.pow(c,2)
    norm.append(math.sqrt(g))


for i in range(len(uniq)):
    for x in wtq[i]:
        for key,val in x.items():
            val = val / norm[i]
            x[key] = val

In [26]:
for i in range(len(uniq)):
    ss = []
    g = 0
    for x in new_wtd[i]:
        for val in x.values():
            if val != 0: ss.append(val)
    for c in ss:
        g = g + math.pow(c,2)
    norms.append(math.sqrt(g))
    

for i in range(len(uniq)):
    for x in new_wtd[i]:
        for key,val in x.items():
            val = val / norms[i]
            x[key] = val

In [27]:
for i in range(len(query)):
    hasilcosine  = []
    for j in range(len(dataStemList)):
        dix  = {}
        ans = []
        for kata in query[i]:
            ind = uniq.index(word)
            for x,y in zip(wtq[ind][i].values(),new_wtd[ind][j].values()):
                ans.append(x*y)
        #print(i+1, j+1, sum(ans))
        if sum(ans)!=0:
            dix['docnum'] = j+1
            dix['score'] = sum(ans)
        if len(dix) != 0: hasilcosine.append(dix)
    rescosine.append(hasilcosine)

In [28]:
for i in range(len(query)):
    rescosine[i] = sorted(rescosine[i], key = lambda x : x['score'], reverse = True)

In [29]:
for i in range(len(query)):
    with open('Tugas1B-{counter}.txt'.format(counter = i+1), 'w') as f:
        f.write('First 10 Documents:\n')
        f.write('Q_ID | DOCNO | SCORE\n')
        if len(rescosine[i]) > 10:
            for x in range(10):
                c = i + 1
                f.write('%s   0   %s   0   %s\n' %(c,docnum[rescosine[i][x]['docnum']-1],rescosine[i][x]['score']))
        else:
            for x in hasil[i]:
                c  = i + 1
                f.write('%s   0   %s   0   %s\n' %(c,docnum[x['docnum']-1],x['score']))